In [1]:
!pip install --upgrade pip
!pip install -r requirements.txt
!pip3 install --upgrade numexpr
!pip install --upgrade keras
# Don't forget to download the ckpt we're fine-tuning

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3
    Uninstalling pip-22.3:
      Successfully uninstalled pip-22.3
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers (from -r requirements.txt (line 23))
  Cloning https://github.com/CompVis/taming-transformers.git (to revision master) to ./src/taming-transformers
  Running command git clone --filter=blob:none --quiet https://github.com/CompVis/taming-transformers.git /home/ubuntu/stable-diffusion/src/taming-transformers
  Resolved https://github.com/CompVis/taming-transformers.git to commit 3ba01b241669f5ade541ce990f7650a3b8

In [ ]:
from datasets import load_dataset
# ds = load_dataset("lambdalabs/pokemon-blip-captions", split="train")

ds = load_dataset("imagefolder", data_dir="/home/ubuntu/mixed-training-images", split="train")
sample = ds[0]
print(sample["text"])
display(sample["image"])

In [ ]:
# Resize dataset to 256, 256
from PIL import Image
import os, sys
import cv2
import numpy as np

def resize():
    path = "/home/ubuntu/people-training/"
    dirs = os.listdir(path)
    for item in dirs:
        if os.path.isfile(path+item) and (item.endswith(".png") or item.endswith(".jpg")):
            im = Image.open(path + item)
            f, e = os.path.splitext(path + item)
            image = im.resize((256, 256), Image.ANTIALIAS)
            image = np.array(image)
            image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
            image = Image.fromarray(np.uint8(image)).convert('RGB')
            image.save(f.replace('people-training', 'people-training-resized') + '.png', 'PNG')


def check_image_channels():
    path = "/home/ubuntu/people-training-resized/"
    dirs = os.listdir(path)
    for file in dirs:
        if os.path.isfile(path + file) and (file.endswith(".png") or file.endswith(".jpg")):
            image = Image.open(path + file)
            image = np.array(image)
            print(image.shape)
            
            
check_image_channels()

In [4]:
import torch

model_path = '/home/ubuntu/sd-v1-4-full-ema.ckpt'

# ckpt = torch.load(model_path)

In [ ]:
!(python main.py \
    -t \
    --base configs/stable-diffusion/art.yaml \
    --scale_lr False \
    --gpus 1 \
    --num_nodes 1 \
    --check_val_every_n_epoch 20 \
    --finetune_from "$model_path" \
    --seed 47 \
    data.params.batch_size=4 \
    lightning.trainer.accumulate_grad_batches=2 \
    data.params.validation.params.n_gpus=1 \
 )

In [ ]:
# Inference with the base model working fine
ckpt_path = 'logs/2022-11-07T04-57-36_pokemon/checkpoints/last.ckpt'

!(python scripts/txt2img.py \
    --prompt 'robotic cat with wings' \
    --outdir 'outputs/generated_pokemon' \
    --H 512 --W 512 \
    --n_samples 4 \
    --config 'configs/stable-diffusion/pokemon.yaml' \
    --ckpt "$ckpt_path")

In [ ]:
!(python scripts/txt2img.py \
    --prompt 'robotic cat with wings' \
    --outdir 'outputs/generated_pokemon' \
    --H 512 --W 512 \
    --n_samples 4 \
    --config 'configs/stable-diffusion/pokemon.yaml' \
    --ckpt "$model_path")